<b> Source COVID World updated everyday at 2:00 AM : </b> <br>
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series

In [ ]:
# %run __init__.ipynb

In [ ]:
# --- STEP1 ---: Get data source & cleaning
START_TIME = time.time()

# -- GITHUB : TIMESERIES (CSSEGISandData/COVID-19/csse_covid_19_time_series)
# -> WORLD_CONFIRMED
URLCONFIRMED = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19'
                '/master/csse_covid_19_data/csse_covid_19_time_series'
                '/time_series_covid19_confirmed_global.csv')
CCONFIRMED = check_url(URLCONFIRMED)
if USE_MONGO:
    bob.mongo.save_df(CCONFIRMED, 'WORLD_CONFIRMED', DB_SRC, True)
CCONFIRMED.to_csv(os.path.join(OUTPUT_FOLDER, 'WORLD_CONFIRMED.csv'), sep=";")

# -> WORLD_DEATHS
URLDEATH = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19'
            '/master/csse_covid_19_data/csse_covid_19_time_series'
            '/time_series_covid19_deaths_global.csv')
CDEATH = check_url(URLDEATH)
if USE_MONGO:
    bob.mongo.save_df(CDEATH, 'WORLD_DEATHS', DB_SRC, True)
CDEATH.to_csv(os.path.join(OUTPUT_FOLDER, 'WORLD_DEATHS.csv'), sep=";")

# -> WORLD_RECOVERED
URLRECOVERY = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19'
               '/master/csse_covid_19_data/csse_covid_19_time_series'
               '/time_series_covid19_recovered_global.csv')
CRECOVERY = check_url(URLRECOVERY)
if USE_MONGO:
    bob.mongo.save_df(CRECOVERY, 'WORLD_RECOVERED', DB_SRC, True)
CRECOVERY.to_csv(os.path.join(OUTPUT_FOLDER, 'WORLD_RECOVERED.csv'), sep=";")

# -> US_CONFIRMED
URLCONFIRMEDUS = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19'
                  '/master/csse_covid_19_data/csse_covid_19_time_series'
                  '/time_series_covid19_confirmed_US.csv')
CCONFIRMEDUS = check_url(URLCONFIRMEDUS)
if USE_MONGO:
    bob.mongo.save_df(CCONFIRMED, 'US_CONFIRMED', DB_SRC, True)
CCONFIRMEDUS.to_csv(os.path.join(OUTPUT_FOLDER, 'US_CONFIRMED.csv'), sep=";")

# -> US_DEATHS
URLDEATHUS = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19'
              '/master/csse_covid_19_data/csse_covid_19_time_series'
              '/time_series_covid19_deaths_US.csv')
CDEATHUS = check_url(URLDEATHUS)
if USE_MONGO:
    bob.mongo.save_df(CDEATH, 'US_DEATHS', DB_SRC, True)
CDEATHUS.to_csv(os.path.join(OUTPUT_FOLDER, 'US_DEATHS.csv'), sep=";")

# -- GITHUB : DAILY REPORT (CSSEGISandData/COVID-19/csse_covid_19_time_series)
# -> DAILY
DATE_INIT = YESTERDAY.strftime('%m-%d-%Y')
DATE_END = YESTERDAY.strftime('%m-%d-%Y')
DATES = pd.date_range(start=DATE_INIT,
                      end=DATE_END,
                      freq='D').strftime('%m-%d-%Y').values.tolist()
FILTER_DATES = pd.date_range(start=DATE_INIT,
                             end=DATE_END,
                             freq='D').strftime('%m/%d/%y').values.tolist()

CDAILY = pd.DataFrame()
for each_date in DATES:
    url_daily = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19'
                 '/master/csse_covid_19_data/csse_covid_19_daily_reports'
                 f'/{each_date}.csv')
    cols_to_rename = {"Country/Region": "Country_Region",
                      "Province/State": "Province_State",
                      "Long_": "Long"}
    tmp_df = check_url(url_daily).rename(index=str, columns=cols_to_rename)
    if tmp_df:
        tmp_df['Confirmed'] = tmp_df['Confirmed'].fillna(0).astype(int)
        tmp_df['Deaths'] = tmp_df['Deaths'].fillna(0).astype(int)
        tmp_df['Recovered'] = tmp_df['Recovered'].fillna(0).astype(int)
        tmp_df['Date'] = pd.to_datetime(each_date,
                                        format='%m-%d-%Y').strftime('%m/%d/%y')
        CDAILY = CDAILY.append(tmp_df)
print("Script execution completed at "
      f"{datetime.now().strftime('%d/%m/%Y %H:%M:%S')}. "
      f"Time: --- {time.time() - START_TIME} secnds ---")

In [ ]:
# with pd.ExcelWriter(output_folder +
#                     'WSR_datasource_' +
#                     yesterday.strftime('%Y%m%d') + '.xlsx') as writer:
#     cConfirmed.to_excel(writer,sheet_name ="WORLD_CONFIRMED")
#     cDeath.to_excel(writer,sheet_name ="WORLD_DEATHS")
#     cRecovery.to_excel(writer,sheet_name ="WORLD_RECOVERED")
#     cDaily.to_excel(writer,sheet_name ="WORLD_DAILY")
#     cConfirmedUS.to_excel(writer,sheet_name ="US_CONFIRMED")
#     cDeathUS.to_excel(writer,sheet_name ="US_DEATHS")

In [ ]:
# -- Melt data source from timeseries to have date in rows
def melt_cleaning(dataf, variable):
    """
    Melt data source from timeseries to have date in rows
    """
    try:
        if 'Province/State' and 'Country/Region' in dataf.columns:
            # Add Country/Region to Province/State if empty
            dataf = dataf.fillna(0)
            dataf.loc[dataf['Province/State'] == 0,
                      'Province/State'] = dataf['Country/Region']
            # Melt and group data
            cols_to_keep = ['Country/Region', 'Province/State']
            cols_to_group = ['Country/Region', 'Province/State', 'Date']
            dataf = (dataf.melt(id_vars=cols_to_keep,
                                value_name=variable,
                                var_name='Date').
                     groupby(cols_to_group,
                             as_index=False).agg({variable: 'sum'}))
            # Data format
            dataf['Date'] = (pd.
                             to_datetime(dataf['Date'],
                                         format='%m/%d/%y').
                             dt.strftime('%m/%d/%y'))

            if variable == 'Confirmed':
                dataf['Deaths'] = 0
                dataf['Recovered'] = 0
            elif variable == 'Deaths':
                dataf['Confirmed'] = 0
                dataf['Recovered'] = 0
            else:
                dataf['Confirmed'] = 0
                dataf['Deaths'] = 0
        else:
            dataf = pd.DataFrame()
    except Exception as expt:
        dataf = pd.DataFrame()
        print(expt.__doc__)
        print(str(expt))
    return dataf


def cols_cleaning(dataf, cols_to_drop):
    """
    doc string
    """
    for each_col in cols_to_drop:
        if each_col in dataf.columns:
            dataf = dataf.drop(each_col, axis=1)

    cols_to_rename = {"Province_State": "Province/State",
                      "Country_Region": "Country/Region"}
    dataf = dataf.rename(index=str, columns=cols_to_rename)
    return dataf


# Timeseries : World Confirmed
COLS_TO_DROP = ['Lat', 'Long']
CCONFIRMED = CCONFIRMED[CCONFIRMED['Country/Region'] != 'US']
DF_CW = melt_cleaning(cols_cleaning(CCONFIRMED, COLS_TO_DROP), 'Confirmed')

# Timeseries : World Deaths
COLS_TO_DROP = ['Lat', 'Long']
CDEATH = CDEATH[CDEATH['Country/Region'] != 'US']
DF_DW = melt_cleaning(cols_cleaning(CDEATH, COLS_TO_DROP), 'Deaths')

# Timeseries : World Recovered
COLS_TO_DROP = ['Lat', 'Long']
DF_RW = melt_cleaning(cols_cleaning(CRECOVERY, COLS_TO_DROP), 'Recovered')

# Timeseries : US Confirmed
COLS_TO_DROP = ['UID', 'iso2', 'iso3', 'code3', 'FIPS',
                'Admin2', 'Lat', 'Long_', 'Combined_Key']
DF_CUS = melt_cleaning(cols_cleaning(CCONFIRMEDUS, COLS_TO_DROP), 'Confirmed')

# Timeseries : US Deaths
COLS_TO_DROP = ['UID', 'iso2', 'iso3', 'code3', 'FIPS',
                'Admin2', 'Lat', 'Long_', 'Combined_Key', 'Population']
DF_DUS = melt_cleaning(cols_cleaning(CDEATHUS, COLS_TO_DROP), 'Deaths')

# Manual input
DF_MANUAL = pd.read_excel(os.path.join(INPUT_FOLDER, 'REF_WSR.xlsx'),
                          sheet_name='DATA_INPUT').fillna(0)
DF_MANUAL['Date'] = pd.to_datetime(DF_MANUAL['Date'],
                                   format='%Y%m%d').dt.strftime('%m/%d/%y')
# DF_MANUAL

In [ ]:
# --- STEP2 ---: Create DB with data source (output = DB_CONCAT)
START_TIME = time.time()


# --- step ---: test
def step2(df1, df2, df3, df4, df5, df6):
    """
    Create DB with data source (output => DB_CONCAT)
    """
    # -- Merge timeseries
    cols_to_group = ['Country/Region', 'Province/State', 'Date']
    dataf = pd.concat([df1, df2, df3, df4, df5, df6],
                      axis=0).groupby(cols_to_group,
                                      as_index=False).agg({'Confirmed': 'sum',
                                                           'Deaths': 'sum',
                                                           'Recovered': 'sum'})
    return dataf.reset_index(drop=True)


DB_CONCAT = step2(DF_CW, DF_DW, DF_RW, DF_CUS, DF_DUS, DF_MANUAL)
print("Script execution completed at "
      f"{datetime.now().strftime('%d/%m/%Y %H:%M:%S')}. "
      f"Time: --- {time.time() - START_TIME} secnds ---")
DB_CONCAT.to_csv(os.path.join(OUTPUT_FOLDER, 'WORLD_DB_CONCAT.csv'), sep=";")
# bob.mongo.save_df(DB_CONCAT, 'WORLD_DB_CONCAT', DB_SRC, True)
# DB_CONCAT

In [ ]:
# --- Check if date is updated
DF_CHECK = DB_CONCAT[DB_CONCAT['Date'] == YESTERDAY.strftime('%m/%d/%y')]
if not DF_CHECK:
    print(f'Data not updated ! DateTime : {now}')
    %stop
else:
    print(f"Data updated, {YESTERDAY} exits in WORLD_DB_CONCAT")

In [ ]:
# --- STEP3 ---: Consolidate and enrich data (output = DB_CONSO)
START_TIME = time.time()
# -> Referentials
REF_CONTINENT = pd.read_excel(os.path.join(INPUT_FOLDER, 'REF_WSR.xlsx'),
                              sheet_name='REF_CONTINENT')


def step3(dataf, ref):
    """
    doc string
    """
    dataf.columns = dataf.columns.str.upper()
    cols_to_rename = {"COUNTRY/REGION": "ENTITY_GROUPS",
                      "PROVINCE/STATE": "ENTITY"}
    dataf = dataf.rename(index=str, columns=cols_to_rename)

    cols_to_rename = {"ENTITY_GROUPS": "ENTITY", "CONTINENT": "ENTITY_GROUPS"}
    df_con = (pd.merge(dataf.drop(['ENTITY'], axis=1),
                       ref.drop(['WORLDMAP'], axis=1),
                       left_on=['ENTITY_GROUPS'],
                       right_on=['COUNTRY_REGION'],
                       how='left').
              rename(index=str, columns=cols_to_rename).
              drop(['COUNTRY_REGION'], axis=1).
              fillna(0))
    df_con.loc[df_con['ENTITY_GROUPS'] == 0,
               'ENTITY_GROUPS'] = 'To be affected'

    cols_to_rename = {"ENTITY_GROUPS": "ENTITY"}
    df_ww = df_con.copy().drop(['ENTITY'],
                               axis=1).rename(index=str,
                                              columns=cols_to_rename)
    df_ww['ENTITY_GROUPS'] = 'WORLDWIDE'

    cols_to_rename = {"ENTITY_GROUPS": "ENTITY"}
    df_tt = df_ww.copy().drop(['ENTITY'],
                              axis=1).rename(index=str,
                                             columns=cols_to_rename)
    df_tt['ENTITY_GROUPS'] = 'WORLDWIDE'

    dataf = pd.concat([dataf, df_con, df_ww, df_tt], axis=0)
    dataf = dataf.groupby(['ENTITY_GROUPS', 'ENTITY', 'DATE'],
                          as_index=False).agg({'CONFIRMED': 'sum',
                                               'DEATHS': 'sum',
                                               'RECOVERED': 'sum'})
    return dataf.reset_index(drop=True)


DB_CONSO = step3(DB_CONCAT, REF_CONTINENT)
print("Script execution completed at "
      f"{datetime.now().strftime('%d/%m/%Y %H:%M:%S')}. "
      f"Time: --- {time.time() - START_TIME} secnds ---")
DB_CONSO.to_csv(os.path.join(OUTPUT_FOLDER + 'WORLD_DB_CONSO.csv'), sep=";")
# if USE_MONGO:
#     bob.mongo.save_df(DB_CONSO, 'WORLD_DB_CONSO', DB_SRC, True)
# DB_CONSO

In [ ]:
# --- STEP4 ---: Calculate KPIs (output = DB_ALL)
START_TIME = time.time()
# -> Referentials
REF_TECH = pd.read_excel(os.path.join(INPUT_FOLDER, 'REF_WSR.xlsx'),
                         sheet_name='PARAM_ALL')


def step4(dataf, ref):
    """
    doc string
    """
    # -- Calc Active + Ratio
    dataf['ACTIVE_CASES'] = (dataf['CONFIRMED'] -
                             dataf['DEATHS'] -
                             dataf['RECOVERED'])
    dataf.loc[:, 'DEATHS_RATIO'] = 0
    dataf.loc[dataf['CONFIRMED'] != 0,
              'DEATHS_RATIO'] = dataf['DEATHS'] / dataf['CONFIRMED'] * 100
    dataf.loc[:, 'RECOVERED_RATIO'] = 0
    dataf.loc[dataf['CONFIRMED'] != 0,
              'RECOVERED_RATIO'] = (dataf['RECOVERED'] /
                                    dataf['CONFIRMED'] *
                                    100)

    # -- Melt KPI in rows
    cols_to_keep = ['ENTITY_GROUPS', 'ENTITY', 'DATE']
    dataf = dataf.melt(id_vars=cols_to_keep,
                       value_name='VALUE',
                       var_name='KPI')
    indexes = dataf.loc[dataf['DATE'] == 0].index
    dataf = dataf.drop(indexes, axis=0)

    # -- Variation vs last day
    # Add fields
    dataf['DATE'] = pd.to_datetime(dataf['DATE'], format='%m/%d/%y')
    dataf['LAST_DAY'] = pd.to_datetime(dataf['DATE'] + timedelta(days=-1))

    # Create new df
    cols_to_rename = {"DATE": "LAST_DAY", 'VALUE': 'VALUE_D-1'}
    df_last = dataf.drop(['LAST_DAY'], axis=1).rename(index=str,
                                                      columns=cols_to_rename)

    # Merge day-1
    cols_to_merge = ['ENTITY_GROUPS', 'ENTITY', 'LAST_DAY', 'KPI']
    dataf = dataf.merge(df_last,
                        on=cols_to_merge,
                        how='left').fillna(0).drop(['LAST_DAY'],
                                                   axis=1)

    # Calc variation in value and %
    dataf['VARV'] = (dataf['VALUE'].astype(float) -
                     dataf['VALUE_D-1'].astype(float))
    dataf.loc[:, 'VARP'] = np.NaN
    dataf.loc[(dataf['KPI'].
               isin(['CONFIRMED',
                     'DEATHS',
                     'RECOVERED',
                     'ACTIVE_CASES'])) &
              dataf['VALUE_D-1'] != 0, 'VARP'] = (dataf['VARV'] *
                                                  100 /
                                                  abs(dataf['VALUE_D-1']))

    # -- Add fields
    dataf['SCENARIO'] = (pd.to_datetime(dataf['DATE'],
                                        format='%m/%d/%y').
                         dt.strftime('%d/%m/%Y'))
    dataf['DATE_ORDER'] = (pd.to_datetime(dataf['SCENARIO'],
                                          format='%d/%m/%Y').
                           dt.strftime('%Y%m%d'))

    # -- Units / Precisions / Sentiments
    dataf['KPI'] = dataf['KPI'].str.replace('_',
                                            ' ').str.lower().str.capitalize()
    dataf = pd.merge(dataf, ref, on=['KPI'], how='left')
    dataf.loc[dataf['KPI'] == 'Deaths ratio', 'KPI'] = "Fatality Rate"
    dataf.loc[dataf['KPI'] == 'Recovered ratio', 'KPI'] = "Recovery Rate"
    return dataf.reset_index(drop=True)


DB_ALL = step4(DB_CONSO, REF_TECH)
print("Script execution completed at "
      f"{datetime.now().strftime('%d/%m/%Y %H:%M:%S')}. "
      f"Time: --- {time.time() - START_TIME} secnds ---")
DB_ALL.to_csv(os.path.join(OUTPUT_FOLDER, 'WORLD_DB_ALL.csv'), sep=";")
# if USE_MONGO:
#     bob.mongo.save_df(DB_ALL, 'WORLD_DB_ALL', DB_SRC, True)
# DB_ALL

In [ ]:
# --- STEP5 ---: Transform DB_ALL to trend data (output = DB_TREND)
START_TIME = time.time()
# -> Referentials
REF_TECH = pd.read_excel(os.path.join(INPUT_FOLDER, 'REF_WSR.xlsx'),
                         sheet_name='PARAM_TREND')


def step5(dataf, ref):
    """
    doc string
    """
    # -- Melt CALC in rows
    cols_to_keep = ['ENTITY_GROUPS', 'ENTITY', 'SCENARIO',
                    'DATE', 'DATE_ORDER', 'KPI']
    dataf = dataf.drop(['VALUE_D-1', 'UNIT_VALUE', 'UNIT_VAR', 'UNIT_VARP',
                        'PRECISION_VALUE', 'PRECISION_VAR', 'PRECISION_VARP'],
                       axis=1).melt(id_vars=cols_to_keep,
                                    value_name='VALUE',
                                    var_name='METRIC')
    dataf['DATE_SCENARIO'] = 'Since beginning'

    df_last14 = get_lastdays(dataf, 14, "Last 14 days")
    df_last30 = get_lastdays(dataf, 30, "Last 30 days")
    dataf = pd.concat([dataf, df_last14, df_last30], axis=0)

    # -- Get units and precisions
    dataf = pd.merge(dataf, ref, on=['KPI', 'METRIC'], how='left')
    dataf.loc[dataf['KPI'] == 'Deaths ratio', 'KPI'] = "Fatality Rate"
    dataf.loc[dataf['KPI'] == 'Recovered ratio', 'KPI'] = "Recovery Rate"
    dataf = dataf.reset_index(drop=True)
    return dataf


DB_TREND = step5(DB_ALL, REF_TECH)
print("Script execution completed at "
      f"{datetime.now().strftime('%d/%m/%Y %H:%M:%S')}. "
      f"Time: --- {time.time() - START_TIME} secnds ---")
DB_TREND.to_csv(os.path.join(OUTPUT_FOLDER, 'WORLD_DB_TREND.csv'), sep=";")
# if use_mongo:
#     bob.mongo.save_df(db_trend,'WORLD_DB_TREND',db_src,True)
# db_trend.head(5)

In [ ]:
# --- STEP6 ---: Check and validate data
# -> CHECK TIMESERIES
# change YESTERDAY format to match with header
YESTERDAY_C = YESTERDAY.strftime('%m/%d/%y')
if YESTERDAY_C[:1] == '0':
    YESTERDAY_C = YESTERDAY_C[1:].replace('/0', '/')


# check if YESTERDAY exist in df
def check_timeseries(date_in, dataf, df_name):
    """
    doc string
    """
    if date_in in dataf.columns:
        check = dataf.agg({date_in: 'sum'}).fillna(0)[0]
    else:
        check = 0
        print(f'Column {date_in} does not exist in {df_name}.\n')
    return check


CHECK_CT = check_timeseries(YESTERDAY_C,
                            CCONFIRMED,
                            'CONFIRMED') + check_timeseries(YESTERDAY_C,
                                                            CCONFIRMEDUS,
                                                            'CONFIRMED')
CHECK_DT = check_timeseries(YESTERDAY_C,
                            CDEATH,
                            'DEATHS') + check_timeseries(YESTERDAY_C,
                                                         CDEATHUS,
                                                         'DEATHS')
CHECK_RT = check_timeseries(YESTERDAY_C, CRECOVERY, 'RECOVERED')
CHECK_AT = CHECK_CT - CHECK_DT - CHECK_RT
if CHECK_CT != 0:
    CHECK_FRT = CHECK_DT / CHECK_CT * 100
    CHECK_RRT = CHECK_RT / CHECK_CT * 100
else:
    CHECK_FRT = 0
    CHECK_RRT = 0


# -> CHECK DAILY
# check if YESTERDAY exist in df
def check_timeseries(date_check, dataf, variable):
    """
    doc string
    """
    if (dataf and
            "Date" in dataf.columns and
            variable in dataf.columns and
            isinstance(date_check, date)):
        check = (dataf[dataf['Date'] == date_check.strftime('%m/%d/%y')].
                 agg({variable: 'sum'})[0])
    else:
        check = 0
        print(f'No data in Daily\n')
    return check


CHECK_CD = check_timeseries(YESTERDAY, CDAILY, 'Confirmed')
CHECK_DD = check_timeseries(YESTERDAY, CDAILY, 'Deaths')
CHECK_RD = check_timeseries(YESTERDAY, CDAILY, 'Recovered')
CHECK_AD = CHECK_CD - CHECK_DD - CHECK_RD
if CHECK_CD:
    CHECK_FRD = CHECK_DD / CHECK_CD * 100
    CHECK_RRD = CHECK_RD / CHECK_CD * 100
else:
    CHECK_FRD = 0
    CHECK_RRD = 0

# -- CHECK : TIMESERIES vs DAILY
CHECK_C = CHECK_CT - CHECK_CD
CHECK_D = CHECK_DT - CHECK_DD
CHECK_R = CHECK_RT - CHECK_RD
CHECK_A = CHECK_AT - CHECK_AD
CHECK_FR = CHECK_FRT - CHECK_FRD
CHECK_RR = CHECK_RRT - CHECK_RRD

print(f'Data check {YESTERDAY}: TimeSeries - Daily')
print(f'Confirmed: {CHECK_CT} - {CHECK_CD} = {CHECK_C}')
print(f'Deaths: {CHECK_DT} - {CHECK_DD} = {CHECK_D}')
print(f'Recovered: {CHECK_RT} - {CHECK_RD} = {CHECK_R}')
print(f'Active cases: {CHECK_AT} - {CHECK_AD} = {CHECK_A}')
print(f'Fatality rate: {CHECK_FRT} - {CHECK_FRD} = {CHECK_FR}')
print(f'Recovery rate: {CHECK_RRT} - {CHECK_RRD} = {CHECK_RR}')